In [1]:
// Load Kotlin Notebook libraries
%use kandy, dataframe

In [2]:
import java.io.File

// Load input and output files
val inputFile = File("resources/FLStartSound.wav")
val processedFile = File("resources/FLStartSound_processed.wav")

### FFT Helpers

In [3]:
// Load FFT helper library and import some commonly used packages
@file:DependsOn("org.quifft:quifft:0.1.1")
import org.quifft.QuiFFT
import org.quifft.output.*

In [75]:
/** Returns the number of frequency bins in an [FFTResult] */
val FFTResult.numBins get(): Int = fftFrames.firstOrNull()?.bins?.size ?: 0

/** Finds and returns the maximum amplitude of each frequency in an FFTResult as a list of [FrequencyBin]s  */
fun FFTResult.binPeaks(): List<FrequencyBin> = List(numBins) {
    fftFrames.maxOfWith(compareBy(FrequencyBin::amplitude)) { frame -> frame.bins[it] }
}

/** Helper method for plotting a frequency spectrum */
fun List<FrequencyBin>.plot() = run {
    val frequency by column(map { it.frequency })
    val amplitude by column(map { it.amplitude })

    dataFrameOf(frequency, amplitude).plot {
        line {
            x(frequency) {
                scale = continuous(transform = Transformation.LOG10)
            }
            y(amplitude) {
                scale = continuous(-65.0..0.0)	
            }
            color(amplitude) {
                scale = continuous(range = Color.GREEN..Color.RED, domain = -65.0..0.0, transform = Transformation.SYMLOG)
            }
        }
        layout {
            size = 700 to 300
        }
    }
}

### Audio Helpers

In [5]:
// Load audio helper library and import some commonly used packages
@file:DependsOn("com.soywiz.korge:korge-core-jvm:5.1.0")
import korlibs.audio.format.*
import korlibs.audio.sound.*

In [6]:
import korlibs.io.util.length

class CustomIndexRangeList<T>(val indexRange: IntRange, init: (Int) -> T) : List<T> {
    private val innerList: MutableList<T> = MutableList(indexRange.length) { init(it.customizeIndex()) }

    override val size: Int by innerList::size

    private fun Int.customizeIndex(): Int = this + indexRange.start
    @Throws(IllegalArgumentException::class)
    private fun Int.normalizeIndex(): Int {
        if (this !in indexRange) throw IllegalArgumentException("Index $this is not within bounds $indexRange")
        return this - indexRange.start
    }

    override operator fun get(index: Int): T = innerList[index.normalizeIndex()]
    operator fun set(index: Int, value: T) { innerList[index.normalizeIndex()] = value }
    override fun isEmpty(): Boolean = innerList.isEmpty()
    override fun iterator(): Iterator<T> = innerList.iterator()
    override fun listIterator(): ListIterator<T> = innerList.listIterator()
    override fun listIterator(index: Int): ListIterator<T> = innerList.listIterator(index.normalizeIndex())
    override fun subList(fromIndex: Int, toIndex: Int): List<T> = innerList.subList(fromIndex.normalizeIndex(), toIndex.normalizeIndex())
    override fun lastIndexOf(element: T): Int = innerList.lastIndexOf(element).customizeIndex()
    override fun indexOf(element: T): Int = innerList.indexOf(element).customizeIndex()
    override fun containsAll(elements: Collection<T>): Boolean = innerList.containsAll(elements)
    override fun contains(element: T): Boolean = innerList.contains(element)
}

In [38]:
import kotlinx.coroutines.runBlocking

fun AudioSamples.toMono() = AudioSamples(1, totalSamples).also { out ->
	for (sampleIndex in 0..<totalSamples) {
		out.setFloat(0, sampleIndex, (0..<channels).map { getFloat(it, sampleIndex) }.average().toFloat())
	}
}

fun AudioSamples.feed(
    feedbackBufferRange: IntRange = 0..0,
    feedforwardBufferRange: IntRange = 0..0,
    transform: (sample: Float, feedbackBuffer: CustomIndexRangeList<Float>, feedfowardBuffer: CustomIndexRangeList<Float>) -> Float,
) = AudioSamples(channels, totalSamples).also { out ->
    for (channel in 0..<channels) {
        for (sampleIndex in 0..<totalSamples) {
            val currentSample = getFloat(channel, sampleIndex)
            val feedbackBuffer = CustomIndexRangeList<Float>(feedbackBufferRange) { indexOffset ->
                (sampleIndex + indexOffset).takeIf { it in 0..<out.totalSamples }?.let { out.getFloat(channel, it) } ?: 0f
            }
            val feedforwardBuffer = CustomIndexRangeList<Float>(feedforwardBufferRange) { indexOffset ->
                (sampleIndex + indexOffset).takeIf { it in 0..<totalSamples }?.let { getFloat(channel, it) } ?: 0f
            }
            out.setFloat(channel, sampleIndex, transform(currentSample, feedbackBuffer, feedforwardBuffer))
        }
    }
}

fun File.applyEffects(outputFile: File, compute: AudioData.() -> AudioData) = runBlocking {
	var data = WAV.decode(readBytes())!!
	outputFile.writeBytes(WAV.encodeToByteArray(data.compute()))
}

### EQ Utils

In [101]:
import korlibs.math.PI2F
import korlibs.math.PIF
import kotlin.properties.Delegates

/**
 * @see <span>Robert Bristow-Johnson's <a href="https://archive.ph/20121220231853/http://www.musicdsp.org/files/Audio-EQ-Cookbook.txt">cookbook formulae for audio EQ biquad filter coefficients</a></span>
 */
fun biquadTransform(
    sample: Float,
    inSample1: Float, inSample2: Float,
    outSample1: Float, outSample2: Float,
    b0: Float, b1: Float, b2: Float,
    a0: Float, a1: Float, a2: Float,
): Float = (b0 / a0) * sample + (b1 / a0) * inSample1 + (b2 / a0) * inSample2 - (a1 / a0) * outSample1 - (a2 / a0) * outSample2

data class Coefficients(
	val b0: Float, val b1: Float, val b2: Float,
	val a0: Float, val a1: Float, val a2: Float,
) {
    companion object {
        val Zero = Coefficients(0f, 0f, 0f, 0f, 0f, 0f)
    }
}

interface EQFilter {
    var sampleRate: Int
    var coefficients: Coefficients
    
    fun calculateCoefficients(sampleRate: Int): Coefficients
}

fun EQFilter.compute(
    sampleRate: Int,
    sample: Float,
    inSample1: Float, inSample2: Float,
    outSample1: Float, outSample2: Float,
): Float {
    if (this.sampleRate != sampleRate) {
        this.sampleRate = sampleRate
        coefficients = calculateCoefficients(sampleRate)
    }
    val (b0, b1, b2, a0, a1, a2) = coefficients
    return biquadTransform(sample, inSample1, inSample2, outSample1, outSample2, b0, b1, b2, a0, a1, a2)
}

data class Peak(val frequency: Float, val bandwidth: Float, val gain: Float) : EQFilter {
    private val A: Float = 10f.pow(gain / 40)

    override var sampleRate: Int = 0
    override var coefficients: Coefficients = Coefficients.Zero
    
    override fun calculateCoefficients(sampleRate: Int): Coefficients {
        val w0 = PI2F * frequency / sampleRate
        val cosw0 = cos(w0)
        val sinw0 = sin(w0)
        val alpha = sinw0 * sinh(ln(2f) / 2 * bandwidth * w0 / sinw0)
        return Coefficients(
			b0 =  1 + alpha * A,
			b1 = -2 * cosw0,
			b2 =  1 - alpha * A,
			a0 =  1 + alpha / A,
			a1 = -2 * cosw0,
			a2 =  1 - alpha / A,
        )
    }
}
data class Notch(val frequency: Float, val bandwidth: Float) : EQFilter {
    override var sampleRate: Int = 0
    override var coefficients: Coefficients = Coefficients.Zero
    
    override fun calculateCoefficients(sampleRate: Int): Coefficients {
        val w0 = PI2F * frequency / sampleRate
        val cosw0 = cos(w0)
        val sinw0 = sin(w0)
        val alpha = sinw0 * sinh(ln(2f) / 2 * bandwidth * w0 / sinw0)
        return Coefficients(
            b0 =  1f,
            b1 = -2 * cosw0,
            b2 =  1f,
            a0 =  1 + alpha,
            a1 = -2 * cosw0,
            a2 =  1 - alpha,
        )
    }
}
data class HighShelf(val frequency: Float, val shelfSlope: Float, val gain: Float) : EQFilter {
    private val A: Float = 10f.pow(gain / 40)

    override var sampleRate: Int = 0
    override var coefficients: Coefficients = Coefficients.Zero
    
    override fun calculateCoefficients(sampleRate: Int): Coefficients {
        val w0 = PI2F * frequency / sampleRate
        val cosw0 = cos(w0)
        val sinw0 = sin(w0)
        val alpha = sinw0 / 2 * sqrt((A + 1 / A) * (1 / shelfSlope - 1) + 2)
        val srqtAalpha2 = 2 * sqrt(A) * alpha
        return Coefficients(
            b0 =      A * ((A + 1) + (A - 1) * cosw0 + srqtAalpha2),
            b1 = -2 * A * ((A - 1) + (A + 1) * cosw0),
            b2 =      A * ((A + 1) + (A - 1) * cosw0 - srqtAalpha2),
            a0 =           (A + 1) - (A - 1) * cosw0 + srqtAalpha2,
            a1 =      2 * ((A - 1) - (A + 1) * cosw0),
            a2 =           (A + 1) - (A - 1) * cosw0 - srqtAalpha2,
        )
    }
}
data class LowShelf(val frequency: Float, val shelfSlope: Float, val gain: Float) : EQFilter {
    private val A: Float = 10f.pow(gain / 40)
    
    override var sampleRate: Int = 0
    override var coefficients: Coefficients = Coefficients.Zero
    
    override fun calculateCoefficients(sampleRate: Int): Coefficients {
        val w0 = PI2F * frequency / sampleRate
        val cosw0 = cos(w0)
        val sinw0 = sin(w0)
        val alpha = sinw0 / 2 * sqrt((A + 1 / A) * (1 / shelfSlope - 1) + 2)
        val srqtAalpha2 = 2 * sqrt(A) * alpha
        return Coefficients(
            b0 =     A * ((A + 1) - (A - 1) * cosw0 + srqtAalpha2),
            b1 = 2 * A * ((A - 1) - (A + 1) * cosw0),
            b2 =     A * ((A + 1) - (A - 1) * cosw0 - srqtAalpha2),
            a0 =          (A + 1) + (A - 1) * cosw0 + srqtAalpha2,
            a1 =    -2 * ((A - 1) + (A + 1) * cosw0),
            a2 =          (A + 1) + (A - 1) * cosw0 - srqtAalpha2,
        )
    }
}
data class LowPass(val frequency: Float, val resonance: Float) : EQFilter {
    override var sampleRate: Int = 0
    override var coefficients: Coefficients = Coefficients.Zero
    
    override fun calculateCoefficients(sampleRate: Int): Coefficients {
        val w0 = PI2F * frequency / sampleRate
        val cosw0 = cos(w0)
        val sinw0 = sin(w0)
        val alpha = sinw0 / (2 * resonance)
        return Coefficients(
            b0 = (1 - cosw0) / 2,
            b1 =  1 - cosw0,
            b2 = (1 - cosw0) / 2,
            a0 =  1 + alpha,
            a1 = -2 * cosw0,
            a2 =  1 - alpha,
        )
    }
}
data class HighPass(val frequency: Float, val resonance: Float) : EQFilter {
    override var sampleRate: Int = 0
    override var coefficients: Coefficients = Coefficients.Zero
    
    override fun calculateCoefficients(sampleRate: Int): Coefficients {
        val w0 = PI2F * frequency / sampleRate
        val cosw0 = cos(w0)
        val sinw0 = sin(w0)
        val alpha = sinw0 / (2 * resonance)
        return Coefficients(
            b0 = (1 + cosw0) / 2,
            b1 = -1 - cosw0,
            b2 = (1 + cosw0) / 2,
            a0 =  1 + alpha,
            a1 = -2 * cosw0,
            a2 =  1 - alpha,
        )
    }
}

fun AudioData.equalize(vararg filters: EQFilter): AudioData = AudioData(rate, 
	filters.fold(samples) { samples, filter ->
		samples.feed(-2..-1, -2..-1) { sample, feedbackBuffer, feedforwardBuffer ->
			filter.compute(rate, sample, feedforwardBuffer[-1], feedforwardBuffer[-2], feedbackBuffer[-1], feedbackBuffer[-2])
		}
	}
)

### Plot initial frequency spectrum

In [77]:
QuiFFT(inputFile).fullFFT().binPeaks().plot()

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="FcXUVj"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"amplitude":[-50.28129241818242,-52.53853395920771,-58.77060655706173,-53.00174023580314,-48.74353162745915,-47.81918871131473,-47.307718603441444,-45.51503478488567,-44.795624603180784,-39.84364670599483,-37.102109070992356,-34.745569705244854,-32.09686877565192,-31.395204715738508,-31.464769302352494,-31.495419002244418,-33.59557924433578,-33.04439051790732,-33.76377415841671,-36.703723182599845,-33.74192027498444,-33.0871411924561,-34.84755974003602,-37.66580301663544,-38.43078553575988,-35.83174439891271,-35.58437724039972,-37.04342102926351,-34.053221558157375,-32.62715761865629,-32.75359002884973,-32.16257647476385,-34.19174494744401,-32.82707775263517,-33.29622080733425,-31.478965381326205,-29.376290685952007,-28.998816711804643,-29.403024774391536,-22.718593300233152,-18.723416236757473,-17.83576320183808,-19.91580326450457,-23.91282665325489,-23.115672164079992,-22.341029283565415,-15.892702711150474,-12.370168160408866,-13.512272697852815,-20.039614024799715,-22.31215416987672,-24.281301798805096,-25.238257801412868,-25.175327881947993,-25.79272715453189,-27.243849506204914,-28.375409660517033,-29.166273062118314,-30.676656736824555,-30.795422029278413,-29.98138561768325,-30.234522059956404,-32.29586508814891,-33.35490278358257,-33.02962186383853,-33.37252038469042,-34.08408107560708,-33.84053974011948,-35.02057097289196,-38.884480151086976,-40.01190950542913,-39.331303243485706,-39.221687039531886,-39.500004275555696,-37.03302227959769,-36.20459777715812,-36.285677757295076,-37.29999103951474,-35.7787165653582,-35.49714228731735,-35.91707550872109,-35.42452793726906,-33.63020332346061,-32.99307438349365,-34.13132183965493,-34.88822369415827,-32.36989398009467,-32.18234667633014,-32.900912146552066,-32.37906317682634,-32.20115233995991,-33.21330046206357,-33.55129692243725,-32.33583917995673,-31.73246580018153,-32.90986037124063,-35.789077289913145,-34.256072886172774,-36.21986305347308,-36.65394752325043,-34.59120987503054,-33.062971959045015,-33.77789506681835,-35.087907651871774,-36.31520898285939,-37.187147464790236,-38.33991756993288,-37.51824886080127,-35.05721724201648,-34.79287436779203,-36.256310598694604,-36.82665751977561,-36.40340596493073,-39.03504597568593,-40.29368514919,-38.898000469699355,-39.934507371502875,-39.74027273347949,-42.259159334347416,-44.74558939996156,-44.89765325644687,-43.876707458061716,-41.72031082105392,-40.03047956801111,-38.8090982858155,-41.58895024384022,-40.70160587893362,-36.61282193718458,-36.33755030860061,-36.568783110954236,-37.04688102308573,-36.03948783096974,-36.89977907857353,-37.429790278949035,-36.376307192668264,-35.03624257268137,-36.47234138388819,-36.44731191018465,-35.91800057221895,-36.46796580363315,-35.5161070496427,-34.514475888863245,-33.891861229331816,-33.39779361044804,-34.060289804462876,-35.476076012755094,-35.22234021864613,-34.813463785934985,-34.79553941250229,-33.96688460707264,-35.316484032209566,-36.19087682762203,-36.67796379336479,-36.02895334186672,-35.4134849741046,-35.76576563884933,-37.7606867067344,-35.41114297901178,-36.30229701300615,-37.06948959412803,-36.029412850352756,-35.15757744534385,-34.817819961689395,-34.500075547149756,-34.1391270715379,-34.571741610146034,-34.68652170134049,-34.71554679310609,-34.847608121624695,-35.16693964241282,-35.74066372168343,-34.685131021012296,-35.85584020321236,-37.654179376179094,-37.50774436416628,-36.392096803886304,-37.02374502446324,-35.94869882610049,-34.43170079880375,-35.464846303054955,-33.177663587739524,-32.26837883571926,-34.571420129404444,-30.338956790097647,-29.672530062780154,-32.8989

### Apply EQ

In [118]:
inputFile.applyEffects(processedFile) {
	equalize(
//		LowPass(1000f, 1f),
//		HighPass(2000f, 1f),
		Notch(500f, 10f),
//      Peak(1000f, .2f, 50f),
//      LowShelf(250f, 1f, -50f), 
//		HighShelf(10000f, 1f, 20f),
	)
}

### Plot filtered frequency spectrum

In [119]:
QuiFFT(processedFile).fullFFT().binPeaks().plot()

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="rfjN3H"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"amplitude":[-41.583939705735105,-43.87023916585579,-45.7211805603867,-46.06113087616387,-45.262144720745205,-47.1173378991272,-46.36771927202932,-46.044812942395396,-48.65581595863,-47.76289736399438,-45.925692078749364,-43.87882788200172,-42.53173024993316,-41.85452199688763,-43.14975846560037,-46.8274753105736,-47.50816678852086,-44.756950667255914,-46.063108867109705,-47.583593562603056,-45.76757842601112,-45.917592278214165,-48.67268713547928,-52.26828522015853,-50.791687165592094,-51.41513089089442,-52.37208485873238,-51.369813153158915,-48.09373959052789,-47.919457823924496,-48.715908201680435,-50.182719510234655,-48.91835727374098,-48.798189960676225,-48.88912142691406,-48.787033232090536,-47.29025109832699,-47.365569103355725,-48.5762238737051,-45.438139966788626,-43.66446734725051,-43.64245553761836,-45.28134201125571,-47.38124383516094,-47.26949374683194,-47.357470777271054,-46.61904481184172,-46.315090387098394,-46.66123980181637,-46.19922495543524,-46.786534770975535,-47.486120015889185,-46.863849887657835,-46.29638963422713,-46.28627565491438,-47.321639872924095,-48.252623636239626,-48.4930955070883,-49.31492911997999,-48.809656325889186,-47.929348711553736,-48.11156525943304,-49.84009022168796,-51.59214520836929,-50.169530580932395,-50.16983211587771,-51.177378580847126,-50.740681332894525,-51.61583793587014,-53.96571150999269,-56.224314430609674,-55.0872657682993,-54.75986577586212,-53.94832673175364,-51.590459600482966,-50.90873331616086,-50.905975756570705,-51.88323024581087,-49.97571412636937,-50.097121480204976,-50.650319162910364,-50.07103386366225,-48.04473772674288,-47.25088197470207,-47.88159616136102,-48.75056446287489,-46.20291376110415,-46.178802348035845,-46.874096827468236,-46.241756125165765,-46.17447510017902,-47.15111311506099,-48.099174889984376,-45.978508962470215,-45.32962251090878,-46.34398561923069,-48.517169079261734,-47.21426485705308,-48.80769071232035,-49.17362182692881,-47.411564733400446,-45.76409030305132,-46.39116083768791,-47.65579761276761,-48.17677889561547,-49.827657356437285,-50.52384348772888,-49.37344733617831,-47.03574347668814,-46.69367237776509,-48.19811304354911,-48.66264125699302,-48.61317613895065,-50.72879853962561,-51.863819270308994,-50.954179137414755,-52.02845153105728,-51.85531943456438,-54.546447058434026,-56.04306294475185,-52.828948175035364,-51.472617182783765,-51.988420195348446,-51.375186122715846,-49.89231841913649,-51.89484388308457,-51.67998410419044,-47.63583391640601,-47.382235197675364,-47.79595362732381,-48.15444384886334,-47.00596319484557,-47.93456743169923,-48.3540326581883,-47.45426856102567,-46.14818993908385,-47.35580232090945,-47.47926837911115,-46.81901360427045,-47.22070731260725,-46.209124701816684,-45.172578243022194,-44.476089710590344,-44.042903166595295,-44.670490799737756,-45.95752938199573,-45.83347457452608,-45.19227232135365,-45.16557818532097,-44.50991138540651,-45.87850484513194,-46.64149794884578,-47.16693902230239,-46.2644982140478,-45.7725713786764,-46.09004478767056,-47.97091516819034,-45.65170239891763,-46.46609975286701,-47.14813797072416,-46.1089834027889,-45.28935298186268,-44.86686304728505,-44.479472974677236,-44.10987664283999,-44.498304651886684,-44.48178974897898,-44.45963913331815,-44.69876870793583,-45.007690430013,-45.400253839436914,-44.328603828846155,-45.546700695810614,-47.485290291567885,-46.97003776650257,-45.967488563280895,-46.60670657869627,-45.382269732661,-44.13876801458632,-44.9633187188016,-42.55696011835556,-41.796740553672684,-43.96241847948778,-39.721009745079996,-39.08311653661731,-42.22044513478758,-